In [3]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;
import os

import nltk
import gensim
from gensim import corpora

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import subprocess
import json
from nltk.stem import WordNetLemmatizer

import gensim
import re  # regular expressions (for playing with the text)
from gensim.parsing.preprocessing import STOPWORDS # common english "stop words" -- a, the, etc.
from gensim.parsing import PorterStemmer
from gensim import corpora, models
import nltk.data

import datetime
from dateutil.parser import parse
import time
import json

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

# Functions that needed to clean the data 

In [5]:
def remove_nonascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

def separate_hashtags(text):
    return set(part[1:] for part in text.split() if part.startswith("#"))
        
def replace_abbrevs(text):
    return re.sub(r'([a-zA-Z])([\'\-\.])(?=[a-zA-Z])', r'\1', text)

def replace_email(text):
    return re.sub(r'[\w\.\-]+@[\w\.\-]+', 'emailtoken', text)

def replace_urls(text):
    """
    Thank you https://www.pythonsheets.com/notes/python-rexp.html :-)
    """
    
    url_exp1 = re.compile(r'''\b(https?:\/\/)? # match http or https
    ([\da-z\.-]+)            # match domain
    \.([a-z\.]{2,6})         # match domain (must be something.something)
    ([\/\w\.\-=\?]*)\/?\b    # match api or file or parameters
    ''', re.X)
    url_exp2 = re.compile(r'''\b(https?:\/\/) # match http or https (must be there)
    ([\da-z\.-]+)            # match domain
    \.?([a-z\.]{2,6})        # match domain (this time may not have the ".something")
    ([\/\w\.\-=\?]*)\/?\b    # match api or file or parameters
    ''', re.X)
    
    return re.sub(url_exp2, 'urltoken', re.sub(url_exp1, 'urltoken', text))

def replace_users(text):
    return re.sub(r'(@\w+)', 'usertagtoken', text)

def replace_hashtags(text):
    return re.sub(r'(#\w+)', 'hashtagtoken', text)

def replace_numbers(text):
    price_exp = re.compile(r"\$(\d*\,){,}\d+\.?\d*")
    pct_exp = re.compile(r'\b(\d*\,){,}\d+\.?\d*\%')
    counter_exp = re.compile(r"(\d*\,){,}\d+\.?\d*(st|nd|rd|th)s?")
    num_exp = re.compile(r"\b(\d*\,){,}\d+\.?\d*\b")
    text = re.sub(price_exp, 'moneytoken', text)
    text = re.sub(pct_exp, 'percenttoken', text)
    text = re.sub(counter_exp, 'countertoken', text)
    return re.sub(num_exp, 'numbertoken', text)

def read_corpus_basic(corp):
    for doc in corp:
        yield [x for x in gensim.utils.simple_preprocess(doc, deacc=True)]

def read_corpus_with_stemming_and_SW_removal(corp):
    lemmatizer = WordNetLemmatizer()
    for doc in corp:
        yield [lemmatizer.lemmatize(x) for x in gensim.utils.simple_preprocess(doc, deacc=True)
                   if x.lower() not in STOPWORDS]

In [ ]:
def encode_docs(tokenizer, max_length, docs):
    # integer encode
    encoded = tokenizer.texts_to_sequences(docs)
    # pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

def isFileAvailable(filename):
    try:
        
        with open(filename, 'r') as test:
            print("File is ready!")
            return True
    except Exception as e:
        
        print("The file is not ready at this moment!" )
        return False

    
def define_model_own():
    model = Sequential()
    model.add(embedding_layer)
    model.add(Conv1D(filters = 32, kernel_size=8, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(12, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    #model.summary()
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model


def classify(tokenizer,nn_model,text):
        
        test_post = []
        test_post.append(text)
        Xpredict = encode_docs(tokenizer,max_length, test_post)
        i = nn_model.predict(Xpredict)
        
        if i[0][0]>0.5:
            return (i[0][0],"relevant")
        else:
            return (i[0][0],"irrlv")


def sentiment_analyzer_scores(sentence):
        score = analyser.polarity_scores(sentence)
        return (score["compound"])

#Code to get json files from cluster
def run_cmd(args_list):
    print('Running system command: {0}'.format(' '.join(args_list)))
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    s_output, s_err = proc.communicate()
    s_return =  proc.returncode
    return s_return, s_output, s_err 

###Getting the Sentiment Label
def sentiment_label(x):
    
    y = "Neutral"
    if x < -0.2:
        y = "Negative"
    elif  x > 0.2:
        y = "Positive"

    else:
        pass


    return (y)

# Training CNN Model

In [7]:
#Data Prep before training the model
dt = pd.read_csv('twitterlabel.csv')

dt['Tokens'] = dt['Tokens'].astype(str).apply(lambda x: x.replace("^", " "))

dt['Label'] = dt['Label'].str.lower()
dt['Label'] = dt['Label'].apply(lambda x: x.replace("e", "")[0:5].strip())

for i in range (0,len(dt['Label'])):
        if dt['Label'][i]!='irrlv':
            dt['Label'][i]='relevant'

#Making csv file ready before re-training
dt1 = dt[list(['Text','Tokens','Label'])]
dt1.to_csv("newtraining.csv",index = False)

dt = pd.read_csv("newtraining.csv")
dt['Tokens'] = dt['Tokens'].astype(str)
y = pd.get_dummies(dt["Label"]).astype(float)

data = dt
train_size = int(len(data) * .8)
 
train_posts = data['Tokens'][:train_size]
train_tags = y[:train_size]
train_files_names = data['Label'][:train_size]
 
test_posts = data['Tokens'][train_size:]
test_tags = y[train_size:]
test_files_names = data['Label'][train_size:]
train_docs = data['Tokens'][:train_size]

max_length = max([len(s.split()) for s in train_docs])

def encode_docs(tokenizer, max_length, docs):
    # integer encode
    encoded = tokenizer.texts_to_sequences(docs)
    # pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

# 2 outputs
num_labels = 2
vocab_size = 20000
batch_size = 100


tokenizer2 = Tokenizer(nb_words=20000)
tokenizer2.fit_on_texts(train_posts)
sequences = tokenizer2.texts_to_sequences(train_posts)

word_index = tokenizer2.word_index
#print('Found %s unique tokens.' % len(word_index))

embeddings_index = {}
f = open('owntwitter.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
#print('Found %s word vectors.' % len(embeddings_index))

#Creating word embedding layer
EMBEDDING_DIM = 1500
MAX_SEQUENCE_LENGTH = max_length

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector



embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

def define_model_own():
    model = Sequential()
    model.add(embedding_layer)
    model.add(Conv1D(filters = 32, kernel_size=8, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(12, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    #model.summary()
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model


Xtrain = encode_docs(tokenizer2, max_length, train_docs)
Xtest = encode_docs(tokenizer2, max_length, test_posts)

# define model
CNN_model_own = define_model_own()
# fit network
CNN_model_own.fit(Xtrain, train_tags, epochs=20, verbose=2)

_, acc = CNN_model_own.evaluate(Xtrain, train_tags, verbose=0)
print('Train Accuracy: %f' % (acc*100))
# evaluate model on test dataset
_, acc = CNN_model_own.evaluate(Xtest, test_tags, verbose=0)
print('Test Accuracy: %f' % (acc*100))


/usr/lib64/python3.4/site-packages/keras/preprocessing/text.py:172: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Epoch 1/20
 - 3s - loss: 0.4048 - acc: 0.8630
Epoch 2/20
 - 3s - loss: 0.2602 - acc: 0.8880
Epoch 3/20
 - 3s - loss: 0.2099 - acc: 0.8940
Epoch 4/20
 - 3s - loss: 0.1910 - acc: 0.9088
Epoch 5/20
 - 3s - loss: 0.1727 - acc: 0.9204
Epoch 6/20
 - 3s - loss: 0.1561 - acc: 0.9306
Epoch 7/20
 - 3s - loss: 0.1324 - acc: 0.9452
Epoch 8/20
 - 3s - loss: 0.1263 - acc: 0.9479
Epoch 9/20
 - 3s - loss: 0.1110 - acc: 0.9537
Epoch 10/20
 - 3s - loss: 0.1038 - acc: 0.9589
Epoch 11/20
 - 3s - loss: 0.0983 - acc: 0.9580
Epoch 12/20
 - 3s - loss: 0.0900 - acc: 0.9637
Epoch 13/20
 - 3s - loss: 0.0881 - acc: 0.9654
Epoch 14/20
 - 3s - loss: 0.0811 - acc: 0.9690
Epoch 15/20
 - 3s - loss: 0.0827 - acc: 0.9662
Epoch 16/20
 - 3s - loss: 0.0782 - acc: 0.9684
Epoch 17/20
 - 3s - loss: 0.0751 - acc: 0.9681
Epoch 18/20
 - 3s - loss: 0.0709 - acc: 0.9697
Epoch 19/20
 - 3s - loss: 0.0715 - acc: 0.9700
Epoch 20/20
 - 3s - loss: 0.0704 - acc: 0.9670
Train Accuracy: 97.171302
Test Accuracy: 76.991150


# Automated Code

In [ ]:
file_hdfs_dir = '/data/atl_sprint_2018/twitter_archive/'
file_name = ''
local_file_name = 'local_twitter_data.json'

while True:
    print('Let the show begins!')
    start_time = time.time()
    
    count = 0
    tweets_data = []
    Time=[]
    tweets=pd.DataFrame()
    tweets_temp=pd.DataFrame()

    #(ret, out, err)= run_cmd(['hadoop', 'fs', '-get', '/data/atl_sprint_2018/twitter_archive/tw20181115*'])


    # Getting the file from cluster
    tmp_file = file_name # this variable will store the name of the latest file in hdfs twitter_archive
    while tmp_file==file_name:
        read_status = os.system('hdfs dfs -ls {} | tail -1 >> hdfs_last_file.txt'.format(file_hdfs_dir))
        with open('hdfs_last_file.txt','r') as lf:
            tmp_file = os.path.basename(lf.read().split(' ')[-1].strip())
            print('Latest available file in twitter_archive: '+tmp_file)
        
        # check if latest file in hdfs matches existing file_name
        if tmp_file==file_name:
            print("New file not AVAILABLE! Waiting 10 min to check again.")
            time.sleep(10*60)
                
    # update filename :-) and download file from hdfs (file is downloaded to local_file_name)
    rm_status = os.system('rm {}'.format(local_file_name)) # remove local twitter file before overwriting :-)
    file_name = tmp_file
    get_status = os.system('hdfs dfs -get {} {}'.format(os.path.join(file_hdfs_dir, file_name), local_file_name))
    # recover date, hour, and seconds (which for some reason are called "i")
    file_time = parse(os.path.splitext(file_name)[0][-13:]) 
    #date1,hour, i = int(file_time.date().strftime('%Y%m%d')), file_time.hour, file_time.minute
    date,hour, i = file_time.date(), file_time.hour, file_time.minute
    date1 = datetime.datetime.strptime(str(file_time.date()),'%Y-%m-%d').strftime('%Y-%m-%d')
    date2 = datetime.datetime.strptime(str(file_time.date()),'%Y-%m-%d').strftime('%Y%m%-d')
#     date1 = '2018-12-06'
#     hour = 23

    #Getting the weekday and hour
    weekday = datetime.datetime.today().weekday()
    train_hour = datetime.datetime.now().hour
    day = datetime.datetime.now().strftime("%A")


######################################################################################################################    
    # Reading the file in python
    try:
        with open(local_file_name, 'r') as f: 
            for line in f:
                try:
                    tweets_data.append(json.loads(line))
                except:
                    continue
        count = count + len(tweets_data)

        tweets_temp=pd.DataFrame()

        tweets_temp['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
        tweets_temp['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
        tweets_temp['id_str'] = list(map(lambda tweet: tweet['id_str'], tweets_data))

        tweets=tweets.append(tweets_temp).reset_index(drop=True)

        #Removing Duplicates
        unq_tweets=tweets.drop_duplicates(subset=['id_str','text','created_at'],keep='first',inplace=False).reset_index(drop=True)    

        
    except Exception as e:
        raise(e)

    if len(unq_tweets) > 50:
        print('Enough Data')

    
        
        #Making Predictions using the trained model

        df_predict = unq_tweets
        df_predict['original_tweets'] = df_predict['text']
        #Clean the csv file before making prediction
        df_predict['text'] = df_predict.text.apply(remove_nonascii)
        df_predict['text'] = df_predict.text.apply(replace_abbrevs)
        df_predict['text'] = df_predict.text.apply(replace_email)
        df_predict['text'] = df_predict.text.apply(replace_urls)
        df_predict['text'] = df_predict.text.apply(replace_numbers)
        df_predict['text'] = df_predict.text.apply(replace_hashtags)
        df_predict['text'] = df_predict.text.apply(replace_users)
        corp2 = list(read_corpus_with_stemming_and_SW_removal(df_predict.text))
        bigrams2 = gensim.models.phrases.Phrases(threshold=50)
        bigrams2.add_vocab(corp2)
        bigram_phraser2 = gensim.models.phrases.Phraser(bigrams2)
        df_predict['Tokens'] = ["^".join(bigram_phraser2[tokens]) for tokens in corp2]
        df_predict['Tokens'] = df_predict['Tokens'].apply(lambda x: x.replace("^", " "))
        df_predict['Tokens'] = df_predict['Tokens'].apply(lambda x: x.replace("_", " "))
        df_predict["Tokens"] = df_predict["Tokens"].str.replace("usertagtoken","")
        df_predict["Tokens"] = df_predict["Tokens"].str.replace("hashtagtoken","")
        df_predict["Tokens"] = df_predict["Tokens"].str.replace("numbertoken","")
        df_predict["Tokens"] = df_predict["Tokens"].str.replace("url","")
        df_predict["Tokens"] = df_predict["Tokens"].str.replace("rt","")

        #Only select the Tokens column for prediction
    #     predict_docs = df_predict['Tokens']

    #     Xpredict = encode_docs(tokenizer2, max_length, predict_docs)

    #     CNN_model_own.predict(Xpredict) 



        classification,confidence=[],[]    
        for t in df_predict['Tokens']:
            conf,result=classify(tokenizer2, CNN_model_own,t)
            classification.append(result)
            confidence.append(conf)



        df_predict['Label'] = pd.Series(classification)  #Add data to the dataframe
        df_predict['Confidence'] = pd.Series(confidence) 

        df_newdata=df_predict[(df_predict['Confidence'] > 0.99) | (df_predict['Confidence'] <= 0.19)].reset_index(drop=True)
        #Converting the predictions back to ire/relev
        #y_predict = []  # array containing the binary classes output.

        # Predicting the output (as interger array) from the x_test.
    #     y_pred_new = CNN_model_own.predict(Xpredict)


    #     for i in y_pred_new:
    #         #print (i)
    #         if i[0] > i[1]:
    #             i[0] = int(1)
    #             i[1] = int(0)
    #             y_predict.append("irrlv")
    #         else:
    #             i[0] = int(0)
    #             i[1] = int(1)
    #             y_predict.append("relevant")


        #Append the label column back to the dataframe
        #df_predict['label'] = y_predict

        #Export the dataframe to the csv file
        #df_predict.to_csv('Twitter_Prediction.csv', index=False)


        if os.path.exists("trainingdata.csv"):
            with open("trainingdata.csv", 'a') as f:
                df_newdata.to_csv(f, header=False,index=False)
        else:
            df_newdata.to_csv("trainingdata.csv",index= False)

        print("Done with storing training data!")




        df_predict.to_csv('Twitter_Prediction.csv', index=False)
        print("Done with Predictions!")


######################################################################################################################
        
        #Topic Modeling
        # Reading the csv file and selecting the relevant tweets rows.
        data = pd.read_csv("Twitter_Prediction.csv")
        data_r = data['Label'] != "irrlv"
        data_rel = data[data_r]
        #data_rel

        # Creating an array of the relevant tweet tokens.
        doc_clean2 = []
        data_rel['Tokens'] = data_rel["Tokens"].astype(str)
        for i in data_rel["Tokens"]:
            #print (i)

            doc_clean2.append(i.split())



        # Topic modelling using LDA

        # Creating the term dictionary of our corpus, where every unique term is assigned an index.  
        dictionary = corpora.Dictionary(doc_clean2)

        # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above. 
        doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean2]

        # Creating the object for LDA model using gensim library
        Lda = gensim.models.ldamodel.LdaModel

        # Running and Training LDA model on the document term matrix
        ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)



        # Getting the Topic Modellng output in the required format.
        pd_array = []



        def get_lda_topics(model, num_topics):
            word_dict = {};
            for i in range(num_topics):
                words = model.show_topic(i, topn = 10);
                #print (i)
                #print ([i[0] for i in words])
                #print ([i[1]*1000 for i in words])
                #print ("--------")


                #print ([i,hour, date, [i[0] for i in words], [i[1]*1000 for i in words]])
                #pd_array.append([i,hour, date, [i[0] for i in words], [i[1]*1000 for i in words]])


                for j in words:
                    #print ([i,hour, date,j[0], j[1]*1000])
                    pd_array.append([i,hour, date1,j[0], j[1]*1000])

        # Calling the function. This is important for appending in the list "pd_array".        
        get_lda_topics(ldamodel, 5)    

        # Creating the dataframe from the list.
        df_topic_modelling = pd.DataFrame(pd_array, columns = ["Topic Number", "Hour", "Date", "Token", "Frequency"])



        # This creates a new csv file and append to it


        if os.path.exists("Twitter_Topic_Modelling_.csv"):
            with open("Twitter_Topic_Modelling_.csv", 'a') as f:
                df_topic_modelling.to_csv(f, header=False, sep = "|")
        else:
            df_topic_modelling.to_csv("Twitter_Topic_Modelling_.csv", sep = "|")

        print('Done with Topic Modeling!')

######################################################################################################################
        
        
        #Sentiment Analyze
        # Vader Sentiment Analyser
        analyser = SentimentIntensityAnalyzer()




        # Getting the list with the required data to convert to the dataframe.
        sent_score = []

        for i in data_rel["text"]:
            sent_score.append([i, date1, hour, sentiment_analyzer_scores(i)])


        # Converting to the dataframe
        df_twitter_sent = pd.DataFrame(sent_score, columns = ["Tweet", "Date", "Hour", "Sentiment"])

        # Adding the original tweet as a column
        data_rel["original_tweets"]
        df_twitter_sent["Original_Tweets"] = data_rel["original_tweets"].values



        # This creates a new csv file and append to it




        if os.path.exists("Twitter_Sentiment.csv"):
            with open("Twitter_Sentiment.csv", 'a') as f:
                df_twitter_sent.to_csv(f, header=False, sep = "|")
        else:
            df_twitter_sent.to_csv("Twitter_Sentiment.csv", sep = "|")

        print("Done with Sentiment Analyze")


######################################################################################################################



        #Labeling the Sentiment Score
        df_twitter_sent["Sentiment_Label"] = df_twitter_sent["Sentiment"].apply(sentiment_label)

        ###Getting the label for certain hour
        volume = df_twitter_sent["Sentiment_Label"].value_counts()
        volume = volume.to_dict()
        sld = (df_twitter_sent.groupby("Sentiment_Label").size()/df_twitter_sent["Sentiment_Label"].count())*100
        sldd = sld.to_dict()

        df_volume = pd.DataFrame(list(volume.items()),columns=['Sentiment_Label', 'Volume'])
        df_twitter_perc = pd.DataFrame(list(sldd.items()),columns=['Sentiment_Label', 'Sentiment_Percentage'])
        df_twitter_perc["Hour"] = hour
        df_twitter_perc["Date"] = date1
        df_twitter_perc["Volume"] = df_volume["Volume"]
        df_twitter_perc = df_twitter_perc[['Date','Hour','Sentiment_Label','Volume','Sentiment_Percentage']]



        # This creates a new csv file



        if os.path.exists("Twitter_Sentiment_Hourly.csv"):
            with open("Twitter_Sentiment_Hourly.csv", 'a') as f:
                df_twitter_perc.to_csv(f, header=False,index=False)
        else:
            df_twitter_perc.to_csv("Twitter_Sentiment_Hourly.csv",index= False)

        print("Done with Sentiment Percentage Hourly")

######################################################################################################################
        
        ###Getting daily sentiment
        df_daily = pd.read_csv("Twitter_Sentiment_Hourly.csv")
        dh = df_daily[df_daily.Date != date2]
        dh.to_csv("Twitter_Sentiment_Hourly.csv",index= False)
        daily1 = df_daily.loc[(df_daily['Date'] == date1)]
        daily1.loc[daily1['Sentiment_Label'] == 'Positive', 'Volume'] = daily1.loc[daily1['Sentiment_Label'] == 'Positive', 'Volume'].sum()
        daily1.loc[daily1['Sentiment_Label'] == 'Neutral', 'Volume'] = daily1.loc[daily1['Sentiment_Label'] == 'Neutral', 'Volume'].sum()
        daily1.loc[daily1['Sentiment_Label'] == 'Negative', 'Volume'] = daily1.loc[daily1['Sentiment_Label'] == 'Negative', 'Volume'].sum()
        daily1 = daily1.drop_duplicates(['Sentiment_Label'],keep='first')
        daily1['Sentiment_Percentage'] = (daily1['Volume']/daily1['Volume'].sum())*100
        daily1["Hour"] = hour
        daily1["Date"] = date1





        # This creates a new csv file
        if os.path.exists("Twitter_Sentiment_Daily.csv"):
            daily = pd.read_csv('Twitter_Sentiment_Daily.csv')
            frame = [daily, daily1]
            new = pd.concat(frame)
            new = new[new.Hour == 23]
            #new = new.drop_duplicates(['Volume'],keep='first')
            #print("Opps, its here!")
    
            new.to_csv("Twitter_Sentiment_Daily.csv",index = False)
        else:
            #print("Creating a new one now!")
            daily1.to_csv("Twitter_Sentiment_Daily.csv",index = False)


    #     # This appends the dataframe to an existing csv file.
    #     daily = pd.read_csv('Twitter_Sentiment_Daily.csv')
    #     frame = [daily, daily1]
    #     new = pd.concat(frame)
    #     new = new[new.Hour == 23]

    #     with open("Twitter_Sentiment_Daily.csv", 'a') as f:
    #         print("Its here!")
    #         new.to_csv(f, header=False,index=False)


        print("Done with Sentiment Percentage Daily")
        
######################################################################################################################

        #Trainning the CNN Model if its on Sunday 10am
    
        if weekday == 6 and train_hour == 10:
            print('Its',day, 'today!')
            print('Time to train the MODEL!')
        
        #Training the model now!
            
#             dt = pd.read_csv('twitterlabel.csv')

#             dt['Tokens'] = dt['Tokens'].astype(str).apply(lambda x: x.replace("^", " "))

#             dt['Label'] = dt['Label'].str.lower()
#             dt['Label'] = dt['Label'].apply(lambda x: x.replace("e", "")[0:5].strip())

#             for i in range (0,len(dt['Label'])):
#                     if dt['Label'][i]!='irrlv':
#                         dt['Label'][i]='relevant'

#             dt1 = dt[list(['Text','Tokens','Label'])]
#             dt1.to_csv("newtraining.csv",index = False)
            
            
            file_exists=False
            file_exists = isFileAvailable('trainingdata.csv')
            if file_exists:
                tp = pd.read_csv("trainingdata.csv")
                tp.columns = ['Text', 'created_at', 'id_str','original_tweets', 'Tokens','Label','Confidence']
                tp1 = tp[list(['Text','Tokens','Label'])]
                with open("newtraining.csv", 'a') as f:
                    tp1.to_csv(f, header=False,index=False)
                dt = pd.read_csv('newtraining.csv')
                dt = dt.drop_duplicates(['Tokens'],keep='first')
            else:
                dt = pd.read_csv('newtraining.csv')



            y = pd.get_dummies(dt["Label"]).astype(float)
            dt['Tokens'] = dt['Tokens'].astype(str)

            data = dt
            train_size = int(len(data) * .8)

            train_posts = data['Tokens'][:train_size]
            train_tags = y[:train_size]
            train_files_names = data['Label'][:train_size]

            test_posts = data['Tokens'][train_size:]
            test_tags = y[train_size:]
            test_files_names = data['Label'][train_size:]
            train_docs = data['Tokens'][:train_size]

            max_length = max([len(s.split()) for s in train_docs])



            # 2 outputs
            num_labels = 2
            vocab_size = 20000
            batch_size = 100


            tokenizer2 = Tokenizer(nb_words=20000)
            tokenizer2.fit_on_texts(train_posts)
            sequences = tokenizer2.texts_to_sequences(train_posts)

            word_index = tokenizer2.word_index
            #print('Found %s unique tokens.' % len(word_index))

            embeddings_index = {}
            f = open('owntwitter.txt')
            for line in f:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            f.close()
            #print('Found %s word vectors.' % len(embeddings_index))

            #Creating word embedding layer
            EMBEDDING_DIM = 1500
            MAX_SEQUENCE_LENGTH = max_length

            embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
            for word, i in word_index.items():
                embedding_vector = embeddings_index.get(word)
                if embedding_vector is not None:
                    # words not found in embedding index will be all-zeros.
                    embedding_matrix[i] = embedding_vector



            embedding_layer = Embedding(len(word_index) + 1,
                                        EMBEDDING_DIM,
                                        weights=[embedding_matrix],
                                        input_length=MAX_SEQUENCE_LENGTH,
                                        trainable=False)




            Xtrain = encode_docs(tokenizer2, max_length, train_docs)
            Xtest = encode_docs(tokenizer2, max_length, test_posts)

            # define model
            CNN_model_own = define_model_own()
            # fit network
            CNN_model_own.fit(Xtrain, train_tags, epochs=20, verbose=2)

            _, acc = CNN_model_own.evaluate(Xtrain, train_tags, verbose=0)
            print('Train Accuracy: %f' % (acc*100))
            # evaluate model on test dataset
            _, acc = CNN_model_own.evaluate(Xtest, test_tags, verbose=0)
            print('Test Accuracy: %f' % (acc*100))
            
            print('Done with TRAINING!')
            print('See you Next Week!')
        

            runtime = time.time() - start_time

            sleep_times = 3600 - runtime

            print("See you in", (sleep_times/60), "minutes")

            
            try:
                time.sleep(sleep_times)
            
            except:
                pass
            
        else:
            print('Today is',day,'!')
            print('Dont feel like training the model today!')
            
            runtime = time.time() - start_time

            sleep_times = 3600 - runtime

            print("See you in", (sleep_times/60), "minutes")
            
            
            try:

                time.sleep(sleep_times)
                
            except:
                pass
            
######################################################################################################################
            
            
    else:
        print('Not Enough data!')
        
        daily = pd.read_csv('Twitter_Sentiment_Daily.csv')
        frame = [daily, daily1]
        new = pd.concat(frame)
        new.to_csv("Twitter_Sentiment_Daily.csv",index = False)
        
        print("Done with daily sentiment")
        
        
        
        runtime = time.time() - start_time

        sleep_times = 3600 - runtime
        
        print('I will run it again', 'in', (sleep_times/60), 'minutes!')
        
        time.sleep(sleep_times)




Let the show begins!
Latest available file in twitter_archive: tw20181208T0120.json
Enough Data
Done with storing training data!
Done with Predictions!


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Done with Topic Modeling!
Done with Sentiment Analyze
Done with Sentiment Percentage Hourly


/usr/lib64/python3.4/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Done with Sentiment Percentage Daily
Today is Saturday !
Dont feel like training the model today!
See you in 59.254659084479016 minutes
Let the show begins!
Latest available file in twitter_archive: tw20181208T0120.json
New file not AVAILABLE! Waiting 10 min to check again.
